# OUTPUT DEMONSTRATION

In [8]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch.nn as nn
import mkl

# Set the number of threads to be used by MKL for parallel execution
mkl.set_num_threads(4)  # Adjust the number of threads as needed

class PlantHealthPredictor:
    def __init__(self):
        self.model = None

    def load_data_from_file(self, file_path):
        # Load data from CSV file
        data = pd.read_csv(file_path)
        X = data.iloc[:, :-1].values  # Features (all columns except the last)
        y = data.iloc[:, -1].values    # Labels (last column)
        return X, y

    def train_model(self, X, y):
        # Convert labels to Long data type
        y_train = torch.tensor(y, dtype=torch.long)

        # Split data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y_train, test_size=0.2, random_state=42)

        # Define neural network model
        self.model = nn.Sequential(
            nn.Linear(X.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 2)  # Output layer with 2 units for binary classification
        )

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)

        # Train the model
        for epoch in range(100):
            optimizer.zero_grad()
            outputs = self.model(torch.tensor(X_train, dtype=torch.float32))
            loss = criterion(outputs, y_train)
            loss.backward()
            optimizer.step()

    def predict(self, sample):
        # Predict plant health for a new sample
        output = self.model(torch.tensor(sample, dtype=torch.float32).unsqueeze(0))  # Add unsqueeze(0) to add batch dimension
        _, predicted = torch.max(output, 1)
        return predicted.item()

    def feedback(self, prediction):
        # Provide feedback based on plant health prediction
        if prediction == 1:
            print("The plant is healthy. Optimal conditions observed.")
        else:
            print("The plant is unhealthy. Attention needed for optimal growth.")

    def optimal_resource_allocation(self, sample):
        # Optimal resource allocation based on plant health indicators
        optimal_actions = []

        # Sensor values in the CSV
        sensor_names = ["Soil Moisture", "Temperature", "Nutrient Levels", "Acidity (pH)",
                        "Pest Activity", "Oxygen Levels", "Manure Requirements", "Weed Presence"]

        # Print sensor values used for prediction
        print("Sensor values used for prediction:")
        for i, value in enumerate(sample):
            print(f"{sensor_names[i]}: {value}")

        # Soil Moisture
        if sample[0] < 30:
            optimal_actions.append("Increase soil moisture by watering.")

        # Temperature
        if sample[1] > 30:
            optimal_actions.append("Provide shade or reduce exposure to direct sunlight.")

        # Nutrient Levels
        if sample[2] < 3:
            optimal_actions.append("Apply fertilizer or nutrient supplements.")

        # Acidity (pH)
        if sample[3] < 6:
            optimal_actions.append("Adjust pH level by adding lime or other amendments.")

        # Pest Activity
        if sample[4] == 1:
            optimal_actions.append("Implement pest control measures.")

        # Oxygen Levels
        if sample[5] < 20:
            optimal_actions.append("Improve ventilation or aeration.")

        # Manure Requirements
        if sample[6] < 5:
            optimal_actions.append("Increase organic matter or apply compost.")

        # Weed Presence
        if sample[7] == 1:
            optimal_actions.append("Remove weeds manually or use herbicides.")

        print("\nOptimal resource allocation:")
        if len(optimal_actions) == 0:
            print("No specific actions recommended based on the current conditions.")
        else:
            for action in optimal_actions:
                print("-", action)

    def save_model(self, file_path):
        # Save the trained model to a file
        torch.save(self.model.state_dict(), file_path)

    def load_model(self, file_path):
        # Load the trained model from a file
        self.model = nn.Sequential(
            nn.Linear(X.shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 2)  # Output layer with 2 units for binary classification
        )
        self.model.load_state_dict(torch.load(file_path))

if __name__ == "__main__":
    # Create plant health predictor
    predictor = PlantHealthPredictor()

    # Load data from file
    X, y = predictor.load_data_from_file("data.csv")

    # Train model
    predictor.train_model(X, y)

    # Sample a random input for prediction
    random_index = np.random.randint(len(X))
    sample = X[random_index]

    # Predict plant health
    prediction = predictor.predict(sample)

    # Provide feedback based on plant health prediction
    predictor.feedback(prediction)

    # Optimal resource allocation
    predictor.optimal_resource_allocation(sample)

The plant is unhealthy. Attention needed for optimal growth.
Sensor values used for prediction:
Soil Moisture: 29.0
Temperature: 24.0
Nutrient Levels: 5.0
Acidity (pH): 6.9
Pest Activity: 1.0
Oxygen Levels: 23.0
Manure Requirements: 4.0
Weed Presence: 1.0

Optimal resource allocation:
- Increase soil moisture by watering.
- Implement pest control measures.
- Increase organic matter or apply compost.
- Remove weeds manually or use herbicides.


In [3]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim

# Load the dataset
data = pd.DataFrame({
    'Soil_Moisture': [42, 21, 31, 53, 38, 49, 29, 43, 34, 51, 30],
    'Temperature': [26, 36, 31, 27, 23, 29, 24, 38, 33, 37, 25],
    'Nutrient_Levels': [5, 4, 7, 8, 6, 3, 5, 2, 9, 4, 8],
    'Acidity': [6.8, 6.6, 6.3, 6.7, 6.1, 6.5, 6.9, 7.2, 7.0, 7.1, 6.4],
    'Pest_Activity': [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0],
    'Oxygen_Levels': [24, 21, 19, 22, 26, 18, 23, 20, 25, 17, 24],
    'Manure_Requirements': [4, 3, 5, 6, 4, 5, 4, 6, 3, 5, 4],
    'Weed_Presence': [0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    'Plant_Health': [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]
})

# Split data into features (X) and target variable (y)
X = data.drop('Plant_Health', axis=1).values
y = data['Plant_Health'].values

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Define neural network model
class PlantHealthModel(nn.Module):
    def __init__(self):
        super(PlantHealthModel, self).__init__()
        self.fc1 = nn.Linear(8, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model
model = PlantHealthModel()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

# Evaluate the model
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print('Accuracy:', accuracy)


Accuracy: 1.0


In [4]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim

# Load the dataset
data = pd.DataFrame({
    'Soil_Moisture': [42, 21, 31, 53, 38, 49, 29, 43, 34, 51, 30],
    'Temperature': [26, 36, 31, 27, 23, 29, 24, 38, 33, 37, 25],
    'Nutrient_Levels': [5, 4, 7, 8, 6, 3, 5, 2, 9, 4, 8],
    'Acidity': [6.8, 6.6, 6.3, 6.7, 6.1, 6.5, 6.9, 7.2, 7.0, 7.1, 6.4],
    'Pest_Activity': [1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0],
    'Oxygen_Levels': [24, 21, 19, 22, 26, 18, 23, 20, 25, 17, 24],
    'Manure_Requirements': [4, 3, 5, 6, 4, 5, 4, 6, 3, 5, 4],
    'Weed_Presence': [0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1],
    'Plant_Health': [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0]
})

# Split data into features (X) and target variable (y)
X = data.drop('Plant_Health', axis=1).values
y = data['Plant_Health'].values

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Define neural network model
class PlantHealthModel(nn.Module):
    def __init__(self):
        super(PlantHealthModel, self).__init__()
        self.fc1 = nn.Linear(8, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Instantiate the model
model = PlantHealthModel()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

# Evaluate the model
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print('Accuracy:', accuracy)


Accuracy: 0.6666666666666666
